In [1]:
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [2]:
import torch.utils.data as Data

batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.4739, -0.7847],
        [-0.5342, -1.2158],
        [ 0.4668,  0.6251],
        [ 0.7588, -0.3758],
        [-1.5564, -1.4937],
        [-1.6195,  0.0659],
        [-0.8987, -0.9158],
        [-0.5953,  1.1358],
        [ 0.5845,  0.1384],
        [ 0.0156,  1.2880]]) tensor([ 5.9317,  7.2674,  3.0018,  6.9783,  6.1687,  0.7336,  5.5312, -0.8674,
         4.8969, -0.1582])


In [4]:
class LinearNet(torch.nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = torch.nn.Linear(n_feature, 1)
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [5]:
# 写法一
net = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = torch.nn.Sequential()
net.add_module('linear', torch.nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = torch.nn.Sequential(OrderedDict([
          ('linear', torch.nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1005, -0.6004]], requires_grad=True)
Parameter containing:
tensor([0.0902], requires_grad=True)


注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入，如果只有单个样本，可使用input.unsqueeze(0)来添加一维.

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

注：如果这里的net是一开始的代码自定义的，那么上面代码会报错，net[0].weight应改为net.linear.weight，bias亦然。因为net[0]这样根据下标访问子模块的写法只有当net是个ModuleList或者Sequential实例时才可以。

In [9]:
loss = torch.nn.MSELoss()

In [10]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

In [13]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 4.169221
epoch 2, loss: 3.740058
epoch 3, loss: 1.191585


In [16]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.7001, -2.7586]], requires_grad=True)
4.2 Parameter containing:
tensor([3.5100], requires_grad=True)
